In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


# テストデータのpath

In [ ]:
# テストデータのpath
dist_path = "rotated_OIbDia08_6_dist(50).csv"
calib_path = "rotated_OIbDia08_6_calib.csv"
spe_path = "OIbDia08_ 06.spe"

nxs_path = "OIbDia08_6_00000.nxs"
poni_path = "Dioptas_lambda_MgS400_CeO2_240511.poni"

# HDF5ファイルの設定

In [ ]:
import h5py

In [ ]:
def show_hdf5_hierarchy(f, indent=0, path=""):
    for key in f.keys():
        print(" " * indent + "/" + key)  # グループ名を出力
        current_path = path + "/" + key
        if isinstance(f[key], h5py.Group): # もしkey先がgroupの場合
            show_hdf5_hierarchy(f[key], indent + 4, path=current_path)  # グループ内のさらに深い階層を出力
        else:
            print(" " * indent + f" --> path:  \"{current_path}\"")
            print(" " * indent + f"       L {type(f[key])}")
            dataset = f[key]
            
            if dataset.shape == (): # スカラー(単一値)の場合
                value = str(dataset[()]) # スカラーの場合の読み取り
            else:
                value = str(f.get(key)[0])
                
            if len(value) > 20:
                print(" " * indent + f"       L {value[:20]}")
            else:
                print(" " * indent + f"       L {value}")



In [ ]:
# calibのテストで作ったやつ
with h5py.File("CalibData.hdf5", "r") as f: 
    show_hdf5_hierarchy(f)

# 全部入れてみる

In [ ]:
# これに入れる
hdf5_name = "hdf5_test.hdf5" # 実際はファイルをまとめてるフォルダ名(OIbDia08_2ndみたいな)にする

# 階層path
data_layer = "data/" # 一応冗長にしてる

name_layer = data_layer + "hdf5_test"

# ここで初期化
with h5py.File(hdf5_name, "w") as f:
    f.create_dataset(name_layer, data="hdf5_test")

In [ ]:

# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)


In [ ]:
from model.Temperature import (
    Laser,
    Spe,
    T_calib,
    T_dist
)

from model.XRD import Nxs

## 温度階層

In [ ]:
# 階層pathを作る
temperature_layer = data_layer + "temperature/"

### Speデータ

In [ ]:
spe = Spe.SpeFile(filepath=spe_path)

In [ ]:
dir(spe)

In [ ]:
# 階層pathを作る
spe_layer = temperature_layer + "spe/"

param_layer = spe_layer + "param/"
arr_layer = spe_layer + "arr/"

# speのparam
spe_path_layer = param_layer + "spe_path"
spe_frame_num_layer = param_layer + "spe_frame_num"
spe_exposure_ms_layer = param_layer + "spe_exposure_ms"
spe_fps_layer = param_layer + "spe_fps"
spe_OD_layer = param_layer + "spe_OD"
spe_date_layer = param_layer + "spe_date"
spe_calibration_date_layer = param_layer + "spe_calibration_date"
# speのarr
spe_img_layer = arr_layer + "spe_img_arr"
spe_wl_layer = arr_layer + "spe_wl_arr"
spe_time_layer = arr_layer + "spe_time_arr"
spe_pixel_layer = arr_layer + "spe_pixel_arr"

In [ ]:
# あると便利そうな配列をつくる
spe.time_arr = np.arange(spe.num_frames) / spe.framerate
spe.pixel_arr = np.arange(spe.roi_height)

In [ ]:
# 書き込み
# これはHDF5モデルに任せる。
with h5py.File(hdf5_name, "a") as f:
    # NOTE データ型は読み出しの時に指定する
    # param
    f.create_dataset(spe_path_layer,             data=spe.filename)
    f.create_dataset(spe_frame_num_layer,        data=spe.num_frames)
    f.create_dataset(spe_exposure_ms_layer,      data=spe.exposure_time)
    f.create_dataset(spe_fps_layer,              data=spe.framerate)
    f.create_dataset(spe_OD_layer,               data=spe.OD)
    f.create_dataset(spe_date_layer,             data=spe.date)
    f.create_dataset(spe_calibration_date_layer, data=spe.calibration_date)
    # arr
    f.create_dataset(spe_img_layer,   data=spe.img)
    f.create_dataset(spe_wl_layer,    data=spe.x_calibration)
    f.create_dataset(spe_time_layer,  data=spe.time_arr)
    f.create_dataset(spe_pixel_layer, data=spe.time_arr)

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)

### calibデータ

In [ ]:
calib = T_calib.T_calib(filepath=calib_path)

In [ ]:
# 階層pathを作る
t_calib_layer = temperature_layer + "calib/"

calib_path_layer = t_calib_layer + "calib_path"
calib_img_layer = t_calib_layer + "calib_img_arr"

In [ ]:
# 配列を作る
# calib_img
calib_img = calib.return_calib_img(frame_num=spe.num_frames)

In [ ]:
# 書き込み
# これはHDF5モデルに任せる。
with h5py.File(hdf5_name, "a") as f:
    f.create_dataset(calib_path_layer, data=calib.filepath_calib)
    f.create_dataset(calib_img_layer, data=calib_img)

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)

### distデータ

In [ ]:
dist = T_dist.T_dist(filepath=dist_path)
dist.set_T_data()

In [ ]:
# 階層pathを作る
t_dist_layer = temperature_layer + "dist/"

t_dist_path_layer = t_dist_layer + "dist_path"
t_dist_arr = t_dist_layer + "dist_arr"

In [ ]:
# 書き込み
# これはHDF5モデルに任せる。
with h5py.File(hdf5_name, "a") as f:
    f.create_dataset(t_dist_path_layer, data=dist.filepath_dist)
    f.create_dataset(t_dist_arr, data=dist.temp_df.values)

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)

### Laserデータ

In [ ]:
# NOTE Linear加熱しか対応してない
laser = Laser.LaserProfile()

In [ ]:
# 階層pathを作る
laser_layer = temperature_layer + "laser/"



In [ ]:
# 配列を作る



In [ ]:
# 書き込み


In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)

## XRD階層

In [ ]:
# 階層path
xrd_layer = data_layer + "xrd/"

### Nxsデータ

In [ ]:
nxs = Nxs.NxsFile(filepath=nxs_path)

In [ ]:
# 階層pathを作る
nxs_layer = xrd_layer + "nxs/"

In [ ]:
# 配列を作る



In [ ]:
# 書き込み



In [ ]:
# 確認
# TODO ここを追加したものだけ確認できるようにする(pathの文字列で判別)。あとできればshapeとか確認したい


### dioptasデータ

In [ ]:
# 階層pathを作る
# TODO 1d-patternとdatによる背景
# TODO いずれこれは無くしたい。nxsでやる



In [ ]:
# 配列を作る



In [ ]:
# 書き込み



In [ ]:
# 確認



# 正しく入れられているかテスト

## 前準備

In [ ]:
# path一覧を取得する
class Hdf5Path:
    def __init__(self, *, hdf: h5py.File):
        self.f = hdf
        self.path_num = 0
        self.path_list = []
        self.get_hdf5_all_path(f=self.f)
        
    def get_hdf5_all_path(self, f, path=""):
        for key in f.keys():
            current_path = path + "/" + key
            if isinstance(f[key], h5py.Group): # もしkey先がgroupの場合
                self.get_hdf5_all_path(f[key], path=current_path)
            else:
                print(f"{self.path_num: <2}: {current_path}")
                self.path_num += 1
                self.path_list.append(current_path)

In [ ]:
def fetch_from_hdf5(hdf: h5py.File, layer_path: str): # クラスにつける
    with h5py.File(hdf, "r") as f:
        dataset = f[layer_path]

        if dataset.shape == (): # スカラー(単一値)の場合
            data = dataset[()] # スカラーの場合の読み取り
        else:
            data = f.get(layer_path)[:]
    return data

In [ ]:
with h5py.File(hdf5_name, "r") as f:
    hdf_path = Hdf5Path(hdf=f)

## テスト

In [ ]:
# Speデータ




In [ ]:
# calibデータ

# ファイル名がspeと違わないか確認
# speの形で確認


In [ ]:
# distデータ

# ファイル名がspeと違わないか確認
# speの形で確認


In [ ]:
# Laserデータはテストしない

In [ ]:
# Nxsデータ

# speと